# COM4509/6509 MLAI - Assignment Part 2 Brief

## Deadline: Friday, December 3, 2021 at 15:00 hrs

## Scope: Sessions 6 to 8

## Number of marks available for Part 2: 25

### Please READ the whole assignment first, before starting to work on it.

### How and what to submit

1. A Jupyter Notebook with the code in all the cells executed, outputs displayed, and code
documented.

2. Name your Notebook as COM4509-6509_Assignment_Part2_Username_XXXXXX.ipynb where XXXXXX is your username such as abc18de.

3. Upload a .zip file to Blackboard before the deadline that contains two Jupyter Notebooks, one for Part 1 and one for Part 2 (COM4509-6509_Assignment_Part1_Username_XXXXXX.ipynb and COM4509-6509_Assignment_Part2_Username_XXXXXX.ipynb)

4. NO DATA UPLOAD: Please do not upload the data files used in this Notebook. We have a copy already. Instead, please use a relative file path in your code (data files under folder ‘data’), as in the lab notebook so that we can run your code smoothly when needed. So ‘./data/’, instead of ‘/User/username/myfiles/mlai/assignment1/’

### Assessment Criteria

1) Being able to build complete, reproducible machine learning pipelines from loading data to evaluating prediction performance.

2) Being able to design different machine learning models to compare/optimise prediction performance.

3) Being able to perform exploratory data analysis to gain insights.

## A. Reproducibility & readability

In [ ]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
%matplotlib inline

torch.manual_seed(210169508) # Set a random seed for reproducibility

## B. Logistic Regression on BreastMNIST [9 marks]

The first version of the MedMNIST, published in ISBI21 (2010.14925.pdf (arxiv.org)). As taken from the paper: BreastMNIST is based on a dataset of 780 breast ultrasound images. It is categorized into 3 classes: normal, benign and malignant originally but in BreastMNIST, the task is simplified into binary classification by combining normal and benign as positive, and classifying them against malignant as negative. The source dataset with a ratio of 7 : 1 : 2 into training, validation and test set. The source images of 1 × 500 × 500 are resized into 1 × 28 × 28.

We aim to train a L2-regularised logistic regression model to classify the two classes in BreastMNIST using the standard train/validation/test split with decent performance, i.e. much better than the chance level at worst.

### B1 Data loading and inspection [3 mark]

Follow instructions at https://github.com/MedMNIST/MedMNIST to download and load the data. Display at least ten images for each class, i.e. at least 20 images, from the training set. Display at least ten images for each class from the validation set, and display at least ten images for each class from the test set.


### B2 Logistic regression [4 marks]

Keep a record of the three metrics M1 to M3 below for the two models below:
* M1) Training accuracy: the prediction accuracy of a trained model on the training dataset.
* M2) Validation accuracy: the prediction accuracy of a trained model on the validation dataset.
* M3) Testing accuracy: the prediction accuracy of a trained model on the test dataset.

a) Using the built-in logistic regression functions in scikit-learn, train a logistic regression model with L2 regularisation on the training set, use the validation set to choose a good regularisation parameter (a hyperparameter) from at least three choices, and test the chosen model on the test set. Report the three metrics M1 to M3 [2 marks]

b) Using PyTorch (see Question 5 of Lab 6), train a logistic regression model with L2 regularisation on the training set, use the validation set to choose a good regularisation parameter (a hyperparameter) from at least three choices, and test the chosen model on the test set. Report the three metrics M1 to M3 [2 marks]

### B3 Performance comparison (2 marks)

a) Summarise each of the three metrics from the two models in B2 using one or more bar graphs. [1 mark]

b) Describe at least two observations interesting to you. [1 mark]